In [79]:
## imports
import pandas as pd
import numpy as np
import re
import requests
import yaml


## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 1. Example 1: no credentials; no wrapper

Site: National Assessment of Education Progress (NAEP)

Documentation: https://www.nationsreportcard.gov/api_documentation.aspx

Base link: https://www.nationsreportcard.gov/DataService/GetAdhocData.aspx 

## 1.1 Query to pull some data

In [80]:
## using their example query of 2011 writing scores separated by gender
## based on here - https://stackoverflow.com/questions/40836749/pythonic-way-of-writing-a-single-line-long-string
## using the ( ) syntax to formulate a long
## string without linebreaks added
example_naep_query = (
'https://www.nationsreportcard.gov/'
'Dataservice/GetAdhocData.aspx?'
'type=data&subject=writing&grade=8&'
'subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2011')


example_naep_query


'https://www.nationsreportcard.gov/Dataservice/GetAdhocData.aspx?type=data&subject=writing&grade=8&subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2011'

In [81]:
## use requests to call the api
naep_resp = requests.get(example_naep_query)
naep_resp
print(type(naep_resp))

## get the json contents of the response 
## here, we're assuming valid response
naep_resp_j = naep_resp.json()
naep_resp_j

## with result, turn it into a dataframe
naep_resp_d = pd.DataFrame(naep_resp_j['result'])
naep_resp_d

<Response [200]>

<class 'requests.models.Response'>


{'status': 200,
 'result': [{'year': 2011,
   'sample': 'R3',
   'yearSampleLabel': '2011',
   'Cohort': 2,
   'CohortLabel': 'Grade 8',
   'stattype': 'MN:MN',
   'subject': 'WRI',
   'grade': 8,
   'scale': 'WRIRP',
   'jurisdiction': 'NP',
   'variable': 'GENDER',
   'variableLabel': 'Gender',
   'varValue': '1',
   'varValueLabel': 'Male',
   'value': 139.099504632971,
   'isStatDisplayable': 1,
   'errorFlag': 0},
  {'year': 2011,
   'sample': 'R3',
   'yearSampleLabel': '2011',
   'Cohort': 2,
   'CohortLabel': 'Grade 8',
   'stattype': 'MN:MN',
   'subject': 'WRI',
   'grade': 8,
   'scale': 'WRIRP',
   'jurisdiction': 'NP',
   'variable': 'GENDER',
   'variableLabel': 'Gender',
   'varValue': '2',
   'varValueLabel': 'Female',
   'value': 158.567104984955,
   'isStatDisplayable': 1,
   'errorFlag': 0}]}

,year,sample,yearSampleLabel,Cohort,CohortLabel,stattype,subject,grade,scale,jurisdiction,variable,variableLabel,varValue,varValueLabel,value,isStatDisplayable,errorFlag
0,2011,R3,2011,2,Grade 8,MN:MN,WRI,8,WRIRP,NP,GENDER,Gender,1,Male,139.099505,1,0
1,2011,R3,2011,2,Grade 8,MN:MN,WRI,8,WRIRP,NP,GENDER,Gender,2,Female,158.567105,1,0


## 1.2 What happens if there's an error in our query?

In [82]:
## here's a query that from the documentation we know
## won't work since i modified year to 2025 which doesnt
## exist in the data
wrong_naep_query = (
'https://www.nationsreportcard.gov/'
'Dataservice/GetAdhocData.aspx?'
'type=data&subject=writing&grade=8&'
'subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2025')

wrong_naep_query

'https://www.nationsreportcard.gov/Dataservice/GetAdhocData.aspx?type=data&subject=writing&grade=8&subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2025'

In [83]:
## use requests to call the api
naep_wrong_resp = requests.get(wrong_naep_query)
naep_wrong_resp

<Response [200]>

In [84]:
## in the case of this particular api,
## the call returns some response but
## when we try to extract the json containing
## status or results, we get in an error
#naep_wrong_resp.json() # uncomment to see error

### 1.2.2 More all-purpose way of allowing remainder of calls to run: try, except

In [85]:
## putting it in a try; except as general error catching
try:
    results = naep_wrong_resp.json()['result']
except Exception as e:
    print('Failed to get result from API due to error:')
    print(e) # or just: pass

Failed to get result from API due to error:
Invalid control character at: line 1 column 289 (char 288)


### 1.2.3 Can usually also find more targeted way but that varies more across APIs

In [86]:
## if we wanted do more specific error catching,
## see that the status == 400 actually appears here
## so could write if else along those lines
naep_wrong_resp.text
naep_resp.text

if "System.Exception" in naep_wrong_resp.text:
    print("NAEP results not found")

'{"status":400,"result": "System.Exception: The query \'SELECT DISTINCT Framework FROM Cycles WHERE Subject=\'WRI\' AND Cohort=2 AND CONVERT(VARCHAR(10),Year)+Sample IN (\'2025R3\')\' did not return exactly 1 framework. Make sure you can trend the years defined for the given subject and cohort.\r\n   at NRCDataService3.GetAdhocData.GetFramework(NDEContext& ndeContext, String subjectCode, List`1 yearSamples, String cohort) in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 1952\r\n   at NRCDataService3.GetAdhocData.PopulateBaseOrchestratorRequest() in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 1713\r\n   at NRCDataService3.GetAdhocData.ConstructRequest_Datapoint() in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 560\r\n   at NRCDataService3.GetAdhocData.Page_Load(Object sender, EventArgs e) in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 136"}'

'{"status":200,"result": [{"year":2011,"sample":"R3","yearSampleLabel":"2011","Cohort":2,"CohortLabel":"Grade 8","stattype":"MN:MN","subject":"WRI","grade":8,"scale":"WRIRP","jurisdiction":"NP","variable":"GENDER","variableLabel":"Gender","varValue":"1","varValueLabel":"Male","value":139.099504632971,"isStatDisplayable":1,"errorFlag":0},{"year":2011,"sample":"R3","yearSampleLabel":"2011","Cohort":2,"CohortLabel":"Grade 8","stattype":"MN:MN","subject":"WRI","grade":8,"scale":"WRIRP","jurisdiction":"NP","variable":"GENDER","variableLabel":"Gender","varValue":"2","varValueLabel":"Female","value":158.567104984955,"isStatDisplayable":1,"errorFlag":0}]}'

NAEP results not found


## Activity 1: writing a function to make multiple, sequential calls

- Say we want to pull the data for grades 4, 8, and 12
- How can we write a function that iterates over a list of those grades and pulls the data for each grade?

**Note**: an ideal function would have arguments for each parameter in the API like subject, subscale, etc. Here we can leave those other parts constant

In [87]:
# your code here
def naep_api_calls(grades: list,
                   baseurl = 'https://www.nationsreportcard.gov/Dataservice/GetAdhocData.aspx?',
                   query_part1 = 'type=data&subject=writing',
                   query_part3 = '&subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2011'):
                       
    '''This function calls the NAEP API to get data on writing scores for the grade requested in 2011.
    
    Parameters:
        baseurl (str): default URL for NAEP or similar API
        query_part1 (str): first batch of arguments
        grades (list of int): what grades to get data on
        query_part3 (str): third batch of arguments
    Returns:
        DataFrame with the results'''
    
    dflist = []
    
    for grade in grades:
        naep_query = baseurl + query_part1 + ('&grade=' + str(grade)) + query_part3
        naep_resp = requests.get(naep_query)
        
        #if "System.Exception" in naep_resp.text:
        #    print(f"NAEP results not found for grade {str(grade)}, please try with grade in range of 1-12")
        #else:
        try:
            naep_resp_df = pd.DataFrame(naep_resp.json()['result'])
            dflist.append(naep_resp_df)
        except Exception as e:
            print("Failed to get result from API for grade {} due to error:".format(str(grade)))
            print(e)
        
    combined_result_df = pd.concat(dflist)
    
    return(combined_result_df)


naep_api_calls(grades = [4,8,12])

Failed to get result from API for grade 4 due to error:
Invalid control character at: line 1 column 289 (char 288)


,year,sample,yearSampleLabel,Cohort,CohortLabel,stattype,subject,grade,scale,jurisdiction,variable,variableLabel,varValue,varValueLabel,value,isStatDisplayable,errorFlag
0,2011,R3,2011,2,Grade 8,MN:MN,WRI,8,WRIRP,NP,GENDER,Gender,1,Male,139.099505,1,0
1,2011,R3,2011,2,Grade 8,MN:MN,WRI,8,WRIRP,NP,GENDER,Gender,2,Female,158.567105,1,0
0,2011,R3,2011,3,Grade 12,MN:MN,WRI,12,WRIRP,NP,GENDER,Gender,1,Male,141.256978,1,0
1,2011,R3,2011,3,Grade 12,MN:MN,WRI,12,WRIRP,NP,GENDER,Gender,2,Female,155.385917,1,0


# 2. Example 2: needs credentials; no wrapper

In [88]:
## load creds- STUDENTS: REPLACE WITH PATHNAME TO YOURS 
with open("06_my_cred_f0055yb.yaml", 'r') as stream:
    try:
        creds = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

## get the key
API_KEY = creds['yelp_api']['api_key']

In [89]:
## use documentation to define what to search
## doc: https://www.yelp.com/developers/documentation/v3/business_search
## write the query 
base_url = "https://api.yelp.com/v3/businesses/search?"
my_name = "restaurants"
my_location = "Hanover,NH,03755"
yelp_genquery = ('{base_url}'
                'term={name}'
                '&location={loc}').format(base_url = base_url,
                name = my_name,
                loc = my_location)

## use requests to call the API; here, we're
## passing it our credentials (structure varies
## by API and telling it to only return 10 results
## (max is 50 at once)
header = {'Authorization': f"Bearer {API_KEY}"}
yelp_genresp = requests.get(yelp_genquery, headers = header)
yelp_genresp

## then, look at structure of response
yelp_genjson = yelp_genresp.json()

<Response [200]>

In [90]:
## example business
yelp_genjson['businesses'][0]

## more automatic way of summarizing but things end up in lists
## within columns for things like categories
yelp_gendf = pd.DataFrame(yelp_genjson['businesses'])
yelp_gendf.head()

{'id': '8ybF6YyRldtZmU9jil4xlg',
 'alias': 'mollys-restaurant-and-bar-hanover',
 'name': "Molly's Restaurant & Bar",
 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/1YkJFic4Czt9b2FsZyOrwQ/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/mollys-restaurant-and-bar-hanover?adjust_creative=nTGI65bBzQQrgBxkgQjgkw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=nTGI65bBzQQrgBxkgQjgkw',
 'review_count': 471,
 'categories': [{'alias': 'tradamerican', 'title': 'American (Traditional)'},
  {'alias': 'burgers', 'title': 'Burgers'},
  {'alias': 'pizza', 'title': 'Pizza'}],
 'rating': 4.0,
 'coordinates': {'latitude': 43.701144, 'longitude': -72.2894249},
 'transactions': ['delivery'],
 'price': '$$',
 'location': {'address1': '43 South Main St',
  'address2': '',
  'address3': '',
  'city': 'Hanover',
  'zip_code': '03755',
  'country': 'US',
  'state': 'NH',
  'display_address': ['43 South Main St', 'Hanover, NH 03755']},
 'phone': '+16036432570',
 'disp

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,8ybF6YyRldtZmU9jil4xlg,mollys-restaurant-and-bar-hanover,Molly's Restaurant & Bar,https://s3-media2.fl.yelpcdn.com/bphoto/1YkJFi...,False,https://www.yelp.com/biz/mollys-restaurant-and...,471,"[{'alias': 'tradamerican', 'title': 'American ...",4.0,"{'latitude': 43.701144, 'longitude': -72.2894249}",[delivery],$$,"{'address1': '43 South Main St', 'address2': '...",+16036432570,(603) 643-2570,1310.466357
1,hf6fxcImjFpx2tN9ty79yA,jesses-restaurant-and-tavern-hanover,Jesse's Restaurant & Tavern,https://s3-media1.fl.yelpcdn.com/bphoto/2iaRs_...,False,https://www.yelp.com/biz/jesses-restaurant-and...,311,"[{'alias': 'steak', 'title': 'Steakhouses'}, {...",4.0,"{'latitude': 43.68421035115572, 'longitude': -...",[delivery],$$,"{'address1': '7 Blue Sky Dr', 'address2': '', ...",+16036434111,(603) 643-4111,2936.478993
2,neBEWvgHNhsXIBImCYx_6A,murphy-s-on-the-green-hanover,Murphy’s on the Green,https://s3-media2.fl.yelpcdn.com/bphoto/76DCQr...,False,https://www.yelp.com/biz/murphy-s-on-the-green...,165,"[{'alias': 'newamerican', 'title': 'American (...",4.0,"{'latitude': 43.702, 'longitude': -72.2894949}",[delivery],$$,"{'address1': '11 S Main St', 'address2': '', '...",+16036434075,(603) 643-4075,1250.030364
3,5WW4g_LRwau29KyjZGLyAA,sawtooth-kitchen-hanover,Sawtooth Kitchen,https://s3-media3.fl.yelpcdn.com/bphoto/61MNG4...,False,https://www.yelp.com/biz/sawtooth-kitchen-hano...,9,"[{'alias': 'chickenshop', 'title': 'Chicken Sh...",4.0,"{'latitude': 43.70158, 'longitude': -72.289641}",[],NaN,"{'address1': '33 S Main St', 'address2': '', '...",+16036435134,(603) 643-5134,1279.242917
4,S4CsbBsFgJkqVxGh9d68Lw,elixir-restaurant-white-river-junction,Elixir Restaurant,https://s3-media4.fl.yelpcdn.com/bphoto/5plTU1...,False,https://www.yelp.com/biz/elixir-restaurant-whi...,249,"[{'alias': 'newamerican', 'title': 'American (...",4.5,"{'latitude': 43.6461408048586, 'longitude': -7...",[delivery],$$,"{'address1': '188 S Main St', 'address2': '', ...",+18022817009,(802) 281-7009,7747.445993


In [91]:
## more data-specific way of summarizing
## we're doing a simple approach and just retaining
## cols that have a simple str structure
## if doing for real, would want to extract things
def clean_yelp_json(one_biz):

    ## restrict to str cols
    d_str = {key:value for key, value in one_biz.items()
             if type(value) == str}
    
    df_str = pd.DataFrame(d_str, index = [d_str['id']])
    return(df_str)

yelp_stronly = [clean_yelp_json(one_b) for one_b in yelp_genjson['businesses']]
yelp_stronly_df = pd.concat(yelp_stronly)

yelp_stronly_df.head(7)


,id,alias,name,image_url,url,price,phone,display_phone
8ybF6YyRldtZmU9jil4xlg,8ybF6YyRldtZmU9jil4xlg,mollys-restaurant-and-bar-hanover,Molly's Restaurant & Bar,https://s3-media2.fl.yelpcdn.com/bphoto/1YkJFi...,https://www.yelp.com/biz/mollys-restaurant-and...,$$,+16036432570,(603) 643-2570
hf6fxcImjFpx2tN9ty79yA,hf6fxcImjFpx2tN9ty79yA,jesses-restaurant-and-tavern-hanover,Jesse's Restaurant & Tavern,https://s3-media1.fl.yelpcdn.com/bphoto/2iaRs_...,https://www.yelp.com/biz/jesses-restaurant-and...,$$,+16036434111,(603) 643-4111
neBEWvgHNhsXIBImCYx_6A,neBEWvgHNhsXIBImCYx_6A,murphy-s-on-the-green-hanover,Murphy’s on the Green,https://s3-media2.fl.yelpcdn.com/bphoto/76DCQr...,https://www.yelp.com/biz/murphy-s-on-the-green...,$$,+16036434075,(603) 643-4075
5WW4g_LRwau29KyjZGLyAA,5WW4g_LRwau29KyjZGLyAA,sawtooth-kitchen-hanover,Sawtooth Kitchen,https://s3-media3.fl.yelpcdn.com/bphoto/61MNG4...,https://www.yelp.com/biz/sawtooth-kitchen-hano...,NaN,+16036435134,(603) 643-5134
S4CsbBsFgJkqVxGh9d68Lw,S4CsbBsFgJkqVxGh9d68Lw,elixir-restaurant-white-river-junction,Elixir Restaurant,https://s3-media4.fl.yelpcdn.com/bphoto/5plTU1...,https://www.yelp.com/biz/elixir-restaurant-whi...,$$,+18022817009,(802) 281-7009
XVGEEIH5rVB2QzW-qywcJw,XVGEEIH5rVB2QzW-qywcJw,base-camp-cafe-hanover,Base Camp Cafe,https://s3-media1.fl.yelpcdn.com/bphoto/p8_YiE...,https://www.yelp.com/biz/base-camp-cafe-hanove...,$$,+16036432007,(603) 643-2007
34j_2nRCVQBTKafJvncZlg,34j_2nRCVQBTKafJvncZlg,pine-restaurant-hanover-2,PINE Restaurant,https://s3-media2.fl.yelpcdn.com/bphoto/rD7w_r...,https://www.yelp.com/biz/pine-restaurant-hanov...,$$$,+16036468000,(603) 646-8000


# Activity 2: pull restaurants in a different location

- Try running a business search query for your hometown or another place by constructing a query similar to `yelp_genquery` but changing the location parameter
- Other endpoints require feeding what's called the business' fusion id into the API. Take an id from `yelp_stronly.id` and use the documentation here to pull the reviews for that business: https://www.yelp.com/developers/documentation/v3/business_reviews
- **Challenge**: generalize the previous step by writing a function that (1) takes a list of business ids as an input, (2) calls the reviews API for each id, (3) returns the results, and (4) rowbinds all results, i.e. turns them into a single, usable DataFrame

In [92]:
my_hometown = "Mattapoisett,MA,02739"
yelp_homequery = ('{base_url}'
                'term={name}'
                '&location={loc}').format(base_url = base_url,
                name = my_name,
                loc = my_hometown)

yelp_home_resp = requests.get(yelp_homequery, headers = header)
yelp_home_resp

## then, look at structure of response
yelp_home_json = yelp_home_resp.json()

yelp_home_df = pd.DataFrame(yelp_home_json['businesses'])
yelp_home_df.head()

<Response [200]>

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,A8KgOloz_6AXspNFvATfrw,turks-seafood-market-and-sushi-bar-mattapoisett,Turk's Seafood Market & Sushi Bar,https://s3-media4.fl.yelpcdn.com/bphoto/WDPSr0...,False,https://www.yelp.com/biz/turks-seafood-market-...,245,"[{'alias': 'seafoodmarkets', 'title': 'Seafood...",4.0,"{'latitude': 41.66372, 'longitude': -70.7881}","[pickup, delivery]",$$,"{'address1': '83 Marion Rd', 'address2': '', '...",+15087583113,(508) 758-3113,1776.918404
1,ncyIZVlBMUXyotDjswNbrA,walrus-and-captain-mattapoisett,Walrus and Captain,https://s3-media2.fl.yelpcdn.com/bphoto/njukjq...,False,https://www.yelp.com/biz/walrus-and-captain-ma...,25,"[{'alias': 'tradamerican', 'title': 'American ...",4.0,"{'latitude': 41.6635695927503, 'longitude': -7...","[pickup, delivery]",NaN,"{'address1': '33 County Rd', 'address2': '', '...",+15087588124,(508) 758-8124,619.783408
2,LGn3uj4F5Xtk29LeMISJxw,rustico-mattapoisett,Rustico,https://s3-media2.fl.yelpcdn.com/bphoto/_lWP7r...,False,https://www.yelp.com/biz/rustico-mattapoisett?...,30,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 41.66292, 'longitude': -70.78946}","[pickup, delivery]",NaN,"{'address1': '62 Marion Rd', 'address2': '', '...",+15087174676,(508) 717-4676,1633.979015
3,0dvKPFYyYiy-O2546UBF1Q,tastebuds-mattapoisett-3,Tastebuds,https://s3-media4.fl.yelpcdn.com/bphoto/X9jq_3...,False,https://www.yelp.com/biz/tastebuds-mattapoiset...,58,"[{'alias': 'newamerican', 'title': 'American (...",4.0,"{'latitude': 41.66185, 'longitude': -70.8187485}",[delivery],$$,"{'address1': '42 Main St', 'address2': '', 'ad...",+15087588524,(508) 758-8524,812.778433
4,HjLiD0XnlLXE-PBbgiDztQ,the-inn-on-shipyard-park-mattapoisett,The Inn On Shipyard Park,https://s3-media3.fl.yelpcdn.com/bphoto/dT7tc5...,False,https://www.yelp.com/biz/the-inn-on-shipyard-p...,100,"[{'alias': 'salad', 'title': 'Salad'}, {'alias...",3.5,"{'latitude': 41.658329, 'longitude': -70.8129196}",[],$$,"{'address1': '13 Water St', 'address2': '', 'a...",+15087584922,(508) 758-4922,448.961377


In [93]:
inn_url = "https://api.yelp.com/v3/businesses/HjLiD0XnlLXE-PBbgiDztQ/reviews?limit=20&sort_by=yelp_sorted"

yelp_inn_reviews_resp = requests.get(inn_url, headers = header)
yelp_inn_reviews_resp

yelp_inn_reviews_json = yelp_inn_reviews_resp.json()

yelp_inn_reviews_df = pd.DataFrame(yelp_inn_reviews_json['reviews'])
yelp_inn_reviews_df.head()

<Response [200]>

,id,url,text,rating,time_created,user
0,lvQFIICS_1Z4syZGmvsqEw,https://www.yelp.com/biz/the-inn-on-shipyard-p...,Try their Bulleit Manhattan!! They even give y...,5,2022-08-18 09:45:28,"{'id': 'YIEQp9tPJALpZYI3ZK3edA', 'profile_url'..."
1,zF2g9qKI2RBpctgkZQYYFg,https://www.yelp.com/biz/the-inn-on-shipyard-p...,We went here for dinner early on a Saturday ev...,4,2021-08-16 15:57:26,"{'id': 'd0p2Gq7E2Q_LDDrcCLG6mQ', 'profile_url'..."
2,hMMrv_giPd_5sZrnvse3vg,https://www.yelp.com/biz/the-inn-on-shipyard-p...,I have been going to the Inn since I was a lit...,5,2022-07-07 08:26:53,"{'id': 'jvkO3lBydG1pXhSyg8bR5w', 'profile_url'..."


In [94]:
def get_reviews(bussiness_ids):
    yelp_reviews_df = pd.DataFrame()
    
    for bussiness_id in bussiness_ids:
        bussiness_url = f"https://api.yelp.com/v3/businesses/{bussiness_id}/reviews?limit=3&sort_by=yelp_sorted"
        yelp_reviews_resp = requests.get(bussiness_url, headers = header)
        yelp_bussiness_reviews_df = pd.DataFrame(yelp_reviews_resp.json()['reviews'])
        yelp_bussiness_reviews_df['bussiness_id'] = bussiness_id
        yelp_reviews_df = pd.concat([yelp_reviews_df, yelp_bussiness_reviews_df])
        
    return yelp_reviews_df
yelp_reviews_df = get_reviews(yelp_home_df['id'][0:6])
yelp_reviews_df.sort_values(by = "bussiness_id")

yelp_bussiness_reviews_join_df = yelp_home_df.merge(yelp_reviews_df, left_on = "id", right_on = "bussiness_id", suffixes=("_bussinessInfo", "_reviewInfo"), how="left")
yelp_bussiness_reviews_join_df

,id,url,text,rating,time_created,user,bussiness_id
0,EIw-STXZLWrVrNYCtI87Tg,https://www.yelp.com/biz/tastebuds-mattapoiset...,Reservation highly encouraged!! Small quaint r...,4,2022-12-06 16:44:41,"{'id': 'umDtPnf-fzdLGvW5z2zLQg', 'profile_url'...",0dvKPFYyYiy-O2546UBF1Q
1,nw_itwsvVsDjQ7oRkmbAKg,https://www.yelp.com/biz/tastebuds-mattapoiset...,"The best around! Service, drinks, and the food...",5,2023-02-07 11:19:24,"{'id': 'ZoWzbvqHE8kOquSqOyGdIQ', 'profile_url'...",0dvKPFYyYiy-O2546UBF1Q
2,1CoUeVfuX6RhZsQ4Qn2h2g,https://www.yelp.com/biz/tastebuds-mattapoiset...,Went for breakfast and we found everything so ...,4,2022-06-02 00:04:07,"{'id': 'syXEj01gks2UV8_bsQ8LOQ', 'profile_url'...",0dvKPFYyYiy-O2546UBF1Q
0,BlEDpHUXfajohWGb-NMekQ,https://www.yelp.com/biz/turks-seafood-market-...,Nothing better than sushi by the sea!\n\nThis ...,5,2022-08-15 18:20:49,"{'id': 'EvVUefSzPHDMKd_ivfXc5g', 'profile_url'...",A8KgOloz_6AXspNFvATfrw
1,hNjGQOWYXM-k2Ki-WTq4Cg,https://www.yelp.com/biz/turks-seafood-market-...,"Very okay entree, the crab crusted haddock was...",4,2022-10-18 16:22:59,"{'id': 'PfEP1I2qqujlKDJHsF4jSA', 'profile_url'...",A8KgOloz_6AXspNFvATfrw
2,AsL-ffMiq16sToxcvpXrdg,https://www.yelp.com/biz/turks-seafood-market-...,Turks has always been one of my favorite local...,3,2022-07-10 13:34:41,"{'id': 'ZvzjhKptDJSSG4yvyosDJQ', 'profile_url'...",A8KgOloz_6AXspNFvATfrw
0,lvQFIICS_1Z4syZGmvsqEw,https://www.yelp.com/biz/the-inn-on-shipyard-p...,Try their Bulleit Manhattan!! They even give y...,5,2022-08-18 09:45:28,"{'id': 'YIEQp9tPJALpZYI3ZK3edA', 'profile_url'...",HjLiD0XnlLXE-PBbgiDztQ
1,zF2g9qKI2RBpctgkZQYYFg,https://www.yelp.com/biz/the-inn-on-shipyard-p...,We went here for dinner early on a Saturday ev...,4,2021-08-16 15:57:26,"{'id': 'd0p2Gq7E2Q_LDDrcCLG6mQ', 'profile_url'...",HjLiD0XnlLXE-PBbgiDztQ
2,hMMrv_giPd_5sZrnvse3vg,https://www.yelp.com/biz/the-inn-on-shipyard-p...,I have been going to the Inn since I was a lit...,5,2022-07-07 08:26:53,"{'id': 'jvkO3lBydG1pXhSyg8bR5w', 'profile_url'...",HjLiD0XnlLXE-PBbgiDztQ
0,C3O6lTVd4jbUQ-74Vu2s6A,https://www.yelp.com/biz/rustico-mattapoisett?...,"Great service, the atmosphere was cozy and cut...",5,2022-11-12 19:14:50,"{'id': 'hdi5VJGx2UkVavWSNbZgeQ', 'profile_url'...",LGn3uj4F5Xtk29LeMISJxw


,id_bussinessInfo,alias,name,image_url,is_closed,url_bussinessInfo,review_count,categories,rating_bussinessInfo,coordinates,...,phone,display_phone,distance,id_reviewInfo,url_reviewInfo,text,rating_reviewInfo,time_created,user,bussiness_id
0,A8KgOloz_6AXspNFvATfrw,turks-seafood-market-and-sushi-bar-mattapoisett,Turk's Seafood Market & Sushi Bar,https://s3-media4.fl.yelpcdn.com/bphoto/WDPSr0...,False,https://www.yelp.com/biz/turks-seafood-market-...,245,"[{'alias': 'seafoodmarkets', 'title': 'Seafood...",4.0,"{'latitude': 41.66372, 'longitude': -70.7881}",...,+15087583113,(508) 758-3113,1776.918404,BlEDpHUXfajohWGb-NMekQ,https://www.yelp.com/biz/turks-seafood-market-...,Nothing better than sushi by the sea!\n\nThis ...,5.0,2022-08-15 18:20:49,"{'id': 'EvVUefSzPHDMKd_ivfXc5g', 'profile_url'...",A8KgOloz_6AXspNFvATfrw
1,A8KgOloz_6AXspNFvATfrw,turks-seafood-market-and-sushi-bar-mattapoisett,Turk's Seafood Market & Sushi Bar,https://s3-media4.fl.yelpcdn.com/bphoto/WDPSr0...,False,https://www.yelp.com/biz/turks-seafood-market-...,245,"[{'alias': 'seafoodmarkets', 'title': 'Seafood...",4.0,"{'latitude': 41.66372, 'longitude': -70.7881}",...,+15087583113,(508) 758-3113,1776.918404,hNjGQOWYXM-k2Ki-WTq4Cg,https://www.yelp.com/biz/turks-seafood-market-...,"Very okay entree, the crab crusted haddock was...",4.0,2022-10-18 16:22:59,"{'id': 'PfEP1I2qqujlKDJHsF4jSA', 'profile_url'...",A8KgOloz_6AXspNFvATfrw
2,A8KgOloz_6AXspNFvATfrw,turks-seafood-market-and-sushi-bar-mattapoisett,Turk's Seafood Market & Sushi Bar,https://s3-media4.fl.yelpcdn.com/bphoto/WDPSr0...,False,https://www.yelp.com/biz/turks-seafood-market-...,245,"[{'alias': 'seafoodmarkets', 'title': 'Seafood...",4.0,"{'latitude': 41.66372, 'longitude': -70.7881}",...,+15087583113,(508) 758-3113,1776.918404,AsL-ffMiq16sToxcvpXrdg,https://www.yelp.com/biz/turks-seafood-market-...,Turks has always been one of my favorite local...,3.0,2022-07-10 13:34:41,"{'id': 'ZvzjhKptDJSSG4yvyosDJQ', 'profile_url'...",A8KgOloz_6AXspNFvATfrw
3,ncyIZVlBMUXyotDjswNbrA,walrus-and-captain-mattapoisett,Walrus and Captain,https://s3-media2.fl.yelpcdn.com/bphoto/njukjq...,False,https://www.yelp.com/biz/walrus-and-captain-ma...,25,"[{'alias': 'tradamerican', 'title': 'American ...",4.0,"{'latitude': 41.6635695927503, 'longitude': -7...",...,+15087588124,(508) 758-8124,619.783408,jQYhQkNHp1b88XvgjFMLbA,https://www.yelp.com/biz/walrus-and-captain-ma...,Great shrimp tacos and cocktails. The service ...,4.0,2022-12-08 16:51:15,"{'id': 'V7jnq6HqmGp4gi3pRjNVqg', 'profile_url'...",ncyIZVlBMUXyotDjswNbrA
4,ncyIZVlBMUXyotDjswNbrA,walrus-and-captain-mattapoisett,Walrus and Captain,https://s3-media2.fl.yelpcdn.com/bphoto/njukjq...,False,https://www.yelp.com/biz/walrus-and-captain-ma...,25,"[{'alias': 'tradamerican', 'title': 'American ...",4.0,"{'latitude': 41.6635695927503, 'longitude': -7...",...,+15087588124,(508) 758-8124,619.783408,OYX2tR0aIJv9Ud3aEGVusg,https://www.yelp.com/biz/walrus-and-captain-ma...,Been here few times for dinner and both times ...,1.0,2023-01-29 08:28:36,"{'id': 'cvc3L2NZ3-QR7iqpkGXfbg', 'profile_url'...",ncyIZVlBMUXyotDjswNbrA
5,ncyIZVlBMUXyotDjswNbrA,walrus-and-captain-mattapoisett,Walrus and Captain,https://s3-media2.fl.yelpcdn.com/bphoto/njukjq...,False,https://www.yelp.com/biz/walrus-and-captain-ma...,25,"[{'alias': 'tradamerican', 'title': 'American ...",4.0,"{'latitude': 41.6635695927503, 'longitude': -7...",...,+15087588124,(508) 758-8124,619.783408,TqgqOEv_1KLR_pjgGKSP9A,https://www.yelp.com/biz/walrus-and-captain-ma...,Delicious!! Have tried this place twice and th...,5.0,2022-08-04 05:46:15,"{'id': '44D6B9yOKoF4MEYXvk38oA', 'profile_url'...",ncyIZVlBMUXyotDjswNbrA
6,LGn3uj4F5Xtk29LeMISJxw,rustico-mattapoisett,Rustico,https://s3-media2.fl.yelpcdn.com/bphoto/_lWP7r...,False,https://www.yelp.com/biz/rustico-mattapoisett?...,30,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 41.66292, 'longitude': -70.78946}",...,+15087174676,(508) 717-4676,1633.979015,C3O6lTVd4jb